# **Training vs Testing** 

When working with machine learning models, splitting your dataset ensures that your model generalizes well to new data. The dataset is typically divided as follows:

- Training Set (60-80%) → Used to train the model.
- Validation Set (10-20%) → Used for hyperparameter tuning.
- Test Set (10-20%) → Used to evaluate final model performance.

## **Basic Split (Train-Test Only)**

Use Case: When you don’t need a validation set (e.g., basic models).

```python
from sklearn.model_selection import train_test_split

# Example dataset
X = [...]  # Features
y = [...]  # Target variable

# Split 80% Train, 20% Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

## **Train, Validation, and Test**

Ensures the test and validation sets are not used during training. The validation set helps with hyperparameter tuning before final testing. To include a validation set, perform two consecutive splits:

```python
# First, split into Train (80%) and Temp (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Second, split Temp into Validation (50%) and Test (50%) → Each 10% of original data
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
```

The **training set** is the data that the model will learn how to make predictions from. 

The **validation set** is the data that will be used during the training phase to evaluate the interim performance of the model, guide the tuning of hyper-parameters, and assist in other model improvement capacities (for example, feature selection). Some common metrics used to calculate the performance of machine learning models are accuracy, recall, precision, and F1-Score. The metric we choose to use will vary depending on our particular use case.

The **test** set is the data that will determine the performance of our final model so we can estimate how our model will fare in the real world. To avoid introducing any bias to the final measurements of performance, we do not want the test set anywhere near the model training or tuning processes. That is why the test set is often referred to as the holdout set.

During model fitting, both the features `X` and the true labels `y` of the training set (`X_train`, `y_train`) are used to learn. When evaluating the performance of the model with the validation (`X_val`, `y_val`) or test (Xtest, ytest) set, we are going to temporarily pretend like we do not know the true label of every observation. If we use the observation features in our validation (`X_val`) or test (`X_test`) sets as inputs to the trained model, we can receive a prediction as output for each observation (`y_pred`). We can now compare each of the true labels (`y_val` or `y_test`) with each of the predicted labels (`y_pred`) and get a quantitative evaluation on the performance of the model.

![image](images/train_test.png)

## **Stratified Splitting**

If the target variable is imbalanced (e.g., 90% Class A, 10% Class B), use stratified sampling to maintain class distribution:

```python
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
```
Best for classification problems where class distribution matters.

## **N-Fold Cross-Validation**

Sometimes our dataset is so small that splitting it into training, validation, and test sets that are appropriate sizes is unfeasible. A potential solution is to perform N-Fold Cross-Validation. While we still first split the dataset into a training and test set, we are going to further split the training set into $N$ chunks. In each iteration (or fold), $N-1$ of the chunks are treated as the training set and 1 of the chunks is treated as the validation set over which the evaluation metrics are calculated.

![image](images/n_fold_cross_val.png)

This process is repeated N times cycling through each chunk acting as the validation set and the evaluation metrics from each fold are averaged. For example, in 10-fold cross-validation, we’ll make the validation set the first 10% of the training set and calculate our evaluation metrics. We’ll then make the validation set the second 10% of the data and calculate these statistics once again. We can do this process 10 times, and every time the validation set will be a different chunk of the data. If we then average all of the accuracies, we will have a better sense of how our model does on average.

In [1]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Define the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Define K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and compute scores
scores = cross_val_score(model, X, y, cv=kf, scoring="accuracy")

# Print results
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean():.4f}")

Cross-validation scores: [1.         0.96666667 0.93333333 0.93333333 0.96666667]
Mean accuracy: 0.9600


### **Stratified K-Fold**

If you have imbalanced classes, use `StratifiedKFold`, which ensures each fold has the same class distribution.

In [2]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=skf, scoring="accuracy")

print(f"Stratified CV scores: {scores}")
print(f"Mean accuracy: {scores.mean():.4f}")

Stratified CV scores: [0.96666667 0.96666667 0.93333333 0.96666667 0.9       ]
Mean accuracy: 0.9467


### **Extra Reading**

https://medium.com/data-science/two-common-pitfalls-to-avoid-when-doing-cross-validation-c68ed79c0e4e

## **Time Series Splitting** 

It’s crucial to split time series data in a way that preserves the chronological order and avoids data leakage. For time series data, you cannot randomly split because future data should not be used to predict the past. Instead, use time-based splitting. 



### **TimeSeriesSplit**

Think of `TimeSeriesSplit` as the reliable timekeeper of your data splits. It divides your data into sequential folds, ensuring each training set is formed from past data and each test set from future data.

```python
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
```

![image](images/time_series_split.png)

### **Sliding/Rolling Window Split**

In the rolling window approach, your model moves forward in time with a fixed-size training window that slides along your dataset. It’s like taking steps into the future while always keeping an eye on the past.

```python
for date in pd.date_range('2021–01–01', '2021–12–31', freq='M'):
    delta = date - pd.offsets.MonthBegin(1)
    train = series.loc[delta:date-pd.offsets.Day(1)]
    valid = series.loc[date:date+pd.offsets.MonthEnd(1)]
```

![image](images/sliding_window_split.png)

### **GroupTimeSeriesSplit**

`GroupTimeSeriesSplit` is a scikit-learn compatible version of time series validation with groups, ideal for non-overlapping groups. This technique ensures that the training and test sets do not overlap, which is crucial for time series data to avoid data leakage.

![image](images/group_time_series_split.png)

## **Population Stability Index (PSI)** 

Often model is trained offline and set to use in production for inferencing. In such scenario, it is required to know when to retrain the model. One of the indication of model needs retraining is change in distribution of input attributes. We call it “Data Drift” as well.

The Population Stability Index (PSI) is a statistical metric used to measure the stability or shift in the distribution of a variable (typically a model's score or feature) between two datasets—commonly the training data and a new dataset (e.g., current population or test data). It helps assess whether a machine learning model continues to perform well over time and ensures the data distribution remains consistent.

$\text{PSI}=\Sigma_{i=1}^{n}(A_i\%-E_i\%)\times\ln(\large\frac{A_i\%}{E_i\%})$

PSI is calculated for each feature across the training and current datasets. During feature selection, PSI can help identify variables that are prone to instability. 


| Score Range | Decile | Scoring% (A) | Training% (B) | A - B | ln(A/B) | PSI |
|---|---|---|---|---|---|---|
| >720 | 1 | 12% | 11% | 1% | 0.09 | 0.001 |
| 671-720 | 2 | 11% | 11% | 0% | 0.00 | 0.000 |
| 641-670 | 3 | 14% | 12% | 2% | 0.15 | 0.003 |
| 611-640 | 4 | 12% | 13% | -1% | -0.08 | 0.001 |
| 581-610 | 5 | 12% | 11% | 1% | 0.09 | 0.001 |
| 551-580 | 6 | 10% | 11% | -1% | -0.10 | 0.001 |
| 521-550 | 7 | 12% | 13% | -1% | -0.08 | 0.001 |
| 491-520 | 8 | 6% | 5% | 1% | 0.18 | 0.002 |
| 451-490 | 9 | 6% | 7% | -1% | -0.15 | 0.002 |
| <451 | 10 | 5% | 6% | -1% | -0.18 | 0.002 |
| | | | | | | **0.013** |

A generic rule to decide on model retraining based on PSI —
- PSI < 0.1 — No change. You can continue using existing model.
- PSI >=0.1 but less than 0.2 — Slight change is required.
- PSI >=0.2 — Significant change is required. Ideally, you should not use this model anymore, retraining is required.


 If PSI indicates significant population drift, it may signal the need for:
- Model recalibration (adjusting parameters for current data trends).
- Full retraining with updated datasets to ensure accuracy and relevance.

PSI plays a vital role in monitoring and maintaining the performance and stability of machine learning models in production. It provides insights into data and model stability, helping banks and financial institutions proactively manage risks, ensure compliance, and maintain high-quality decision-making.


# **Extra Reading**

https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right
